# Segmentation model  for analyzing brain MRI data using a segmentation model for ischemic stroke.
The model is trained on a data set of 250 head MRIs labeled with ischemic stroke.


In [1]:
import nibabel as nib
import scipy
import napari
import numpy as np
import os
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import torch
import random
import torch.nn as nn
from torchvision import transforms
import segmentation_models_pytorch as smp
import time

# 1. Custom PyTroch dataset for training data

In [34]:
class ISLES_dataset(Dataset):
    def __init__(self, dataset_dir, transform=None):
        self.dataset_dir = dataset_dir
        self.transform = transform
        self.image_layers = []
        self.mask_layers = []
        for example_case in range(1, 220):
            # images paths
            dwi_path = os.path.join(dataset_dir, 'rawdata', 'sub-strokecase{}'.format('%04d' %example_case), 'ses-0001','dwi',
                                   'sub-strokecase{}_ses-0001_dwi.nii.gz'.format("%04d" % example_case))
            dwi_path = self.make_path_right(dwi_path)
            mask_path = dwi_path.replace('rawdata', 'derivatives').replace('dwi', 'msk').replace('msk', '', 1)
            dwi_image = nib.load(dwi_path).get_fdata().transpose(2, 0, 1)
            mask_image = nib.load(mask_path).get_fdata().transpose(2, 0, 1)
            self.image_layers.extend(dwi_image)
            self.mask_layers.extend(mask_image)
            
    @staticmethod
    def make_path_right(path):
        path = path.replace('\\','/')
        return path
    
    @staticmethod
    def NormalizeTensor(data):
        diff = torch.max(data) - torch.min(data)
        if diff==0:
            return torch.zeros_like(data)
        return (data - torch.min(data)) / (torch.max(data) - torch.min(data))
    
    # augmentation, torchvision.transform doesn't work well on custom segmentation dataset
    def apply_transform(self, image, mask):
        
        # Transform to tensor
        image = transforms.functional.to_tensor(image)
        mask = transforms.functional.to_tensor(mask)
    
        # Resize
        resize = transforms.Resize(size=(128, 128), antialias=None)
        image = resize(image)
        mask = resize(mask)


        # Random horizontal flipping
        if random.random() > 0.5:
            image = transforms.functional.hflip(image)
            mask = transforms.functional.hflip(mask)

        # Random vertical flipping
        if random.random() > 0.5:
            image = transforms.functional.vflip(image)
            mask = transforms.functional.vflip(mask)

        # Random rotation
        degrees = [0, 90, 180, 270]
        angle = random.choice(degrees)
        image = transforms.functional.rotate(image, angle)
        mask = transforms.functional.rotate(mask, angle)
        
        return image, mask
        
    def __len__(self):
        return len(self.mask_layers)
    
    def __getitem__(self, idx):
        
        image_layer = self.image_layers[idx]
        mask_layer = self.mask_layers[idx]
        
        image_layer, mask_layer = self.apply_transform(image_layer, mask_layer)
        image_layer = self.NormalizeTensor(image_layer)
        
        
        return image_layer, mask_layer
        
            


In [35]:
dataset = ISLES_dataset('ISLES-2022/')

In [36]:
train_loader = DataLoader(dataset, batch_size=1)

# Custom PyTorch dataset for validation data

In [51]:
class ISLES_dataset_valid(Dataset):
    def __init__(self, dataset_dir, transform=None):
        self.dataset_dir = dataset_dir
        self.transform = transform
        self.image_layers = []
        self.mask_layers = []
        for example_case in range(221, 251):
            # images paths
            dwi_path = os.path.join(dataset_dir, 'rawdata', 'sub-strokecase{}'.format('%04d' %example_case), 'ses-0001','dwi',
                                   'sub-strokecase{}_ses-0001_dwi.nii.gz'.format("%04d" % example_case))
            dwi_path = self.make_path_right(dwi_path)
  
            mask_path = dwi_path.replace('rawdata', 'derivatives').replace('dwi', 'msk').replace('msk', '', 1)
            dwi_image = nib.load(dwi_path).get_fdata().transpose(2, 0, 1)
            mask_image = nib.load(mask_path).get_fdata().transpose(2, 0, 1)
            self.image_layers.extend(dwi_image)
            self.mask_layers.extend(mask_image)
            
    @staticmethod
    def make_path_right(path):
        path = path.replace('\\','/')
        return path
    
    @staticmethod
    def NormalizeTensor(data):
        diff = torch.max(data) - torch.min(data)
        if diff==0:
            return torch.zeros_like(data)
        return (data - torch.min(data)) / (torch.max(data) - torch.min(data))
    
    def apply_transform(self, image, mask):
        
        # Transform to tensor
        image = transforms.functional.to_tensor(image)
        mask = transforms.functional.to_tensor(mask)
        
        # Resize
        resize = transforms.Resize(size=(128, 128), antialias=None)
        image = resize(image)
        mask = resize(mask)

        # Random horizontal flipping
        if random.random() > 0.5:
            image = transforms.functional.hflip(image)
            mask = transforms.functional.hflip(mask)

        # Random vertical flipping
        if random.random() > 0.5:
            image = transforms.functional.vflip(image)
            mask = transforms.functional.vflip(mask)

        # Random rotation
        degrees = [0, 90, 180, 270]
        angle = random.choice(degrees)
        image = transforms.functional.rotate(image, angle)
        mask = transforms.functional.rotate(mask, angle)
        
        return image, mask
        
    def __len__(self):
        return len(self.mask_layers)
    
    def __getitem__(self, idx):
        
        image_layer = self.image_layers[idx]
        mask_layer = self.mask_layers[idx]
        
        image_layer, mask_layer = self.apply_transform(image_layer, mask_layer)
        image_layer = self.NormalizeTensor(image_layer)
        
        
        return image_layer, mask_layer
        

In [52]:
valid_dataset = ISLES_dataset_valid('ISLES-2022/')

In [53]:
valid_loader = DataLoader(valid_dataset, batch_size=1)

## Unet

In [22]:
class UNet(nn.Module):

    def __init__(self, num_classes):
        super(UNet, self).__init__()
        self.num_classes = num_classes

        # Image reduction
        self.down_conv_11 = self.conv_block(in_channels=1,
                                            out_channels=64)
        self.down_conv_12 = nn.MaxPool2d(kernel_size=2,
                                         stride=2)
        self.down_conv_21 = self.conv_block(in_channels=64,
                                            out_channels=128)
        self.down_conv_22 = nn.MaxPool2d(kernel_size=2,
                                         stride=2)
        self.down_conv_31 = self.conv_block(in_channels=128,
                                            out_channels=256)
        self.down_conv_32 = nn.MaxPool2d(kernel_size=2,
                                         stride=2)
        self.down_conv_41 = self.conv_block(in_channels=256,
                                            out_channels=512)
        self.down_conv_42 = nn.MaxPool2d(kernel_size=2,
                                         stride=2)
        
        self.middle = self.conv_block(in_channels=512, out_channels=1024)
        
        
        
        # image enhancement
        self.up_conv_11 = nn.ConvTranspose2d(in_channels=1024, out_channels=512,
                                             kernel_size=3, stride=2,
                                             padding=1, output_padding=1)
        self.up_conv_12 = self.conv_block(in_channels=1024,
                                          out_channels=512)
        self.up_conv_21 = nn.ConvTranspose2d(in_channels=512, out_channels=256,
                                             kernel_size=3, stride=2,
                                             padding=1, output_padding=1)
        self.up_conv_22 = self.conv_block(in_channels=512,
                                          out_channels=256)
        self.up_conv_31 = nn.ConvTranspose2d(in_channels=256, out_channels=128,
                                             kernel_size=3, stride=2,
                                             padding=1, output_padding=1)
        self.up_conv_32 = self.conv_block(in_channels=256,
                                          out_channels=128)
        self.up_conv_41 = nn.ConvTranspose2d(in_channels=128, out_channels=64,
                                             kernel_size=3, stride=2,
                                             padding=1, output_padding=1)
        self.up_conv_42 = self.conv_block(in_channels=128,
                                          out_channels=64)
        
        self.output = nn.Conv2d(in_channels=64, out_channels=num_classes,
                                kernel_size=3, stride=1,
                                padding=1)
#         self.softmax = nn.Softmax(dim=1)
    
    @staticmethod
    def conv_block(in_channels, out_channels):
        block = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,
                      out_channels=out_channels,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=out_channels),
            nn.Dropout2d(p=0.3),
            nn.Conv2d(in_channels=out_channels,
                      out_channels=out_channels,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=out_channels),
            nn.Dropout2d(p=0.3))
        return block
    
    @staticmethod
    def crop_tensor(target_tensor, tensor):
        target_size = target_tensor.size()[2]
        tensor_size = tensor.size()[2]
        delta = tensor_size - target_size
        delta = delta // 2

        return tensor[:,:, delta:tensor_size-delta, delta:tensor_size-delta]


    def forward(self, X):
        # left
        x1 = self.down_conv_11(X) # [-1, 64, 256, 256]
        x2 = self.down_conv_12(x1) # [-1, 64, 128, 128]
        x3 = self.down_conv_21(x2) # [-1, 128, 128, 128]
        x4 = self.down_conv_22(x3) # [-1, 128, 64, 64]
        x5 = self.down_conv_31(x4) # [-1, 256, 64, 64]
        x6 = self.down_conv_32(x5) # [-1, 256, 32, 32]
        x7 = self.down_conv_41(x6) # [-1, 512, 32, 32]
        x8 = self.down_conv_42(x7) # [-1, 512, 16, 16]
        
        middle_out = self.middle(x8) # [-1, 1024, 16, 16]

        # right
        x = self.up_conv_11(middle_out) # [-1, 512, 32, 32]
        y = self.crop_tensor(x, x7)
        x = self.up_conv_12(torch.cat((x, y), dim=1)) # [-1, 1024, 32, 32] -> [-1, 512, 32, 32]
        
        x = self.up_conv_21(x) # [-1, 256, 64, 64]
        y = self.crop_tensor(x, x5)
        x = self.up_conv_22(torch.cat((x, y), dim=1)) # [-1, 512, 64, 64] -> [-1, 256, 64, 64]
        
        x = self.up_conv_31(x) # [-1, 128, 128, 128]
        y = self.crop_tensor(x, x3)
        x = self.up_conv_32(torch.cat((x, y), dim=1)) # [-1, 256, 128, 128] -> [-1, 128, 128, 128]
        
        x = self.up_conv_41(x) # [-1, 64, 256, 256]
        y = self.crop_tensor(x, x1)
        x = self.up_conv_42(torch.cat((x, y), dim=1)) # [-1, 128, 256, 256] -> [-1, 64, 256, 256]
        
        output = self.output(x) # [-1, num_classes, 256, 256]
#         output = self.softmax(output)

        return output

In [23]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [24]:
Umodel = UNet(num_classes=1).to(device)

# Model train

In [41]:

optimizer = torch.optim.Adam(Umodel.parameters())

In [42]:

criterion = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)

In [56]:
# The model was trained on 150 epochs, so not to show all of them on GitHub, I will show only 5 of them
Umodel = torch.load('best_model.pth')

In [55]:
epochs = 5


epoch_losses = []
best_valid_loss = float('inf')
for epoch in range(epochs):
    start_time = time.time()
    epoch_loss = []
    valid_loss = []
    Umodel.train() 
    for batch_idx, (data, labels) in enumerate(train_loader):
        
        data, labels = data.float().to(device), labels.float().to(device)        
        
        optimizer.zero_grad()
        outputs = Umodel(data)                
        
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss.append(loss.item())
        
    Umodel.eval()
    for batch_idx, (data, labels) in enumerate(valid_loader): ###########
        
        data, labels = data.float().to(device), labels.float().to(device)
        outputs = Umodel(data)
        loss = criterion(outputs, labels)
        valid_loss.append(loss.item())
        
    print(f'Epoch{epoch+1} - train_loss: {np.mean(epoch_loss):.4f} | valid_loss:{np.mean(valid_loss):.4f}')
    end_time = time.time()
    if epoch ==0:
        print(f'Spend time for 1 epoch: {end_time - start_time} sec')
        
    if np.mean(valid_loss) < best_valid_loss:
        best_valid_loss = np.mean(valid_loss)
        torch.save(Umodel, 'best_model.pth')
        print('Model saved')
        
    epoch_losses.append(epoch_loss)

Epoch1 - train_loss: 0.1175 | valid_loss:0.1777
Spend time for 1 epoch: 41.94370174407959 sec
Model saved
Epoch2 - train_loss: 0.1192 | valid_loss:0.1761
Model saved
Epoch3 - train_loss: 0.1145 | valid_loss:0.1765
Epoch4 - train_loss: 0.1113 | valid_loss:0.1777
Epoch5 - train_loss: 0.1083 | valid_loss:0.1764


# Inference

In [69]:
model = torch.load('best_model.pth')

In [70]:
 def make_path_right(path):
        path = path.replace('\\','/')
        return path
    
def NormalizeTensor(data):
    diff = torch.max(data) - torch.min(data)
    if diff==0:
        return torch.zeros_like(data)
    return (data - torch.min(data)) / (torch.max(data) - torch.min(data))

isles_data_dir = 'ISLES-2022/'
example_case = 231

# images paths
dwi_path = os.path.join(isles_data_dir, 'rawdata', 'sub-strokecase{}'.format('%04d' %example_case), 'ses-0001','dwi',
                       'sub-strokecase{}_ses-0001_dwi.nii.gz'.format("%04d" % example_case))
dwi_path = make_path_right(dwi_path)

mask_path = dwi_path.replace('rawdata', 'derivatives').replace('dwi', 'msk').replace('msk', '', 1)

# Load image data.
dwi_image = nib.load(dwi_path).get_fdata().transpose(2, 0, 1)

mask_image = nib.load(mask_path).get_fdata().transpose(2, 0, 1)

In [71]:
image = torch.Tensor(dwi_image)

i=0
for s in image:
    image[i] = NormalizeTensor(s)
    i+=1
    
resize = transforms.Resize(size=(112, 112), antialias=None)
image = resize(image)
image =  image.unsqueeze(1)
shape_of_file = image.shape[0]
result = np.zeros((shape_of_file, 1, 112, 112))

In [72]:
%%time
i=0
for m in image:
    m = m.unsqueeze(0)
    m = m.float()
    output = model(m.to(device))
    preds = torch.sigmoid(output) > 0.5
    result[i] = preds.detach().cpu().numpy()
    i+=1

CPU times: total: 625 ms
Wall time: 605 ms


# Result

In [75]:
from IPython.display import HTML

HTML("""
<div style="display:flex">
    <div style="flex:1;padding-right:5px">
        <img src="prediction.gif">
        <div style="text-align:center">Prediction</div>
    </div>
    <div style="flex:1;padding-left:5px">
        <img src="Ground_truth.gif">
        <div style="text-align:center">Ground Truth</div>
    </div>
</div>
""")